# Observing RNN PSTH
This notebook does similar thing as VirtualRat.ipynb by loading weights and testing performance.
The purporse of this notebook is to pick sample PSTH from RNNs

In [ ]:
import numpy as npp
import minpy.numpy as np
import matplotlib.pyplot as plt
import minpy
minpy.set_global_policy('only_numpy')

from VirtualRatModel import VirtualRatModel
from VirtualRatSolver import VirtualRatSolver
from VirtualRatBox import VirtualRatBox
from VirtualRat import VirtualRat
from dataProcessFunctions import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (8.0, 6.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# auto-reloading external modules
%load_ext autoreload
%autoreload 2

In [ ]:
which_to_choose = load_weights("time_steps_for_3d-0.85.pkl")
rats = []
trial_window = 3

for i in range(len(which_to_choose)):
    ratname = "VirtualRat"+str(which_to_choose[i][0])
    try:
        loaded_params = load_weights("/scratch/xl1066/VirtualRat/publication/TrainingTime/trainedTrainingTime-"+\
                                     str(which_to_choose[i][0])+"-"+str(which_to_choose[i][1]*100)+".pkl")
        print ratname
    except IOError:
        continue
    np.random.seed(which_to_choose[i][0])
    npp.random.seed(which_to_choose[i][0])
    box = VirtualRatBox(mode="alternative",length=1000,block_size=30)
    test_X, test_y = box.X, box.y
    model = VirtualRatModel()
    rat = VirtualRat(model,ratname = ratname, trial_window = trial_window)
    rats.append(rat)
    solver = VirtualRatSolver(model, box) # Dummmy solver in order to load weights.
    np.random.seed(which_to_choose[i][0])
    npp.random.seed(which_to_choose[i][0])
    solver.init()
    solver.load_params(loaded_params)
    probs = rat.predict(test_X, test_y)
    draw_3d(rat.p2a_prob*100, rat.a2p_prob*100, trial_window = trial_window, fixed_size= False)
    rat.compute_PETH_ROC()
    rat.add_prediction_history()

In [ ]:
PETH(rats[-3],switch=False, filename="PETH_block.pkl", legend = False)